# Import Libraries

In [ ]:
!pip install gensim

In [ ]:
!pip install pyldavis

In [ ]:
!pip install --upgrade numpy
!pip install --upgrade pandas
!pip install --upgrade gensim

  Using cached numpy-2.2.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
Using cached numpy-2.2.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.4 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gensim 4.3.3 requires numpy<2.0,>=1.18.5, but you have numpy 2.2.5 which is incompatible.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.2.3 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 2.2.5 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.2.5 which is incompatible.
  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using 

In [ ]:
!pip install keybert

In [ ]:
import numpy as np
import pandas as pd
import string
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import spacy

from gensim.corpora import Dictionary
from gensim.models import TfidfModel
from gensim.models import LdaMulticore
from gensim.models.coherencemodel import CoherenceModel
import multiprocessing
import pyLDAvis
import pyLDAvis.gensim


In [ ]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [ ]:
# Load spaCy for lemmatization
nlp = spacy.load('en_core_web_sm')

# Load Data

In [ ]:
pd.set_option('max_colwidth', None)

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/vjavaly/Baruch-CIS-9665/refs/heads/main/data/NIPS_papers_1500.csv')

In [ ]:
df.shape

(1500, 6)

In [ ]:
df.head(3)

year  \
0  1997   
1  2007   
2  2017   

                                                                                                  title  \
0  Independent Component Analysis for Identification of Artifacts in Magnetoencephalographic Recordings   
1                       Near-Maximum Entropy Models for Binary Neural Representations of Natural Images   
2                     Nearest-Neighbor Sample Compression: Efficiency, Consistency, Infinite Dimensions   

  event_type  \
0        NaN   
1        NaN   
2     Poster   

                                                                                                        pdf_name  \
0  1466-independent-component-analysis-for-identification-of-artifacts-in-magnetoencephalographic-recordings.pdf   
1                       3336-near-maximum-entropy-models-for-binary-neural-representations-of-natural-images.pdf   
2                        6755-nearest-neighbor-sample-compression-efficiency-consistency-infinite-dimensions.pdf   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   abstract  \
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          Abstract Missing   
1                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          Abstract Missing   
2  We examine the Bayes-consistency of a recently proposed 1-nearest-neighbor-based multiclass learning algorithm. This algorithm is derived from sample compression bound

In [ ]:
df.isna().sum()

,0
year,0
title,0
event_type,1000
pdf_name,0
abstract,0
paper_text,0


Bin data

In [ ]:
bins = [1986, 1996, 2006, 2017]  # Shift lower edge down by 1 to include 1987
labels = ['1987–1996', '1997–2006', '2007–2017']

# Apply pd.cut with both ends inclusive
df['Year_Range'] = pd.cut(df['year'], bins=bins, labels=labels, right=True, include_lowest=True)
df['Year_Range'].value_counts()

,count
Year_Range,
2007–2017,843
1997–2006,396
1987–1996,261


In [ ]:
bins = [1986, 1997, 2007, 2017]  # Extend edges to ensure inclusivity
labels = ['1987–1997', '1998–2007', '2008–2017']

df['Year_Range'] = pd.cut(df['year'], bins=bins, labels=labels, right=True, include_lowest=True)
df['Year_Range'].value_counts()

,count
Year_Range,
2008–2017,799
1998–2007,400
1987–1997,301


In [ ]:
df1=df[df['Year_Range']=='1987–1997']
df2=df[df['Year_Range']=='1998–2007']
df3=df[df['Year_Range']=='2008–2017']


## Create New Dataframes

> Add blockquote



In [ ]:
df1_combined= df1['title'] + ' ' + df1['paper_text']
df2_combined= df2['title'] + ' ' + df2['paper_text']
df3_combined= df3['title'] + ' ' + df3['paper_text']

# Preprocess Text

In [ ]:
def preprocess(text):
    if isinstance(text, str):
        # Convert to lowercase
        text = text.lower()

        # Replace newline and multiple whitespaces with a single space
        text = re.sub(r'\s+', ' ', text)

        # Remove digits
        text = re.sub(r'\d+', '', text)

        # Remove punctuation
        text = text.translate(str.maketrans('', '', string.punctuation))

        # Tokenize
        tokens = word_tokenize(text)

        # Remove stopwords
        stop_words = set(stopwords.words('english'))
        tokens = [word for word in tokens if word not in stop_words]

        # Keep tokens with more than 2 characters
        tokens = [word for word in tokens if len(word) > 2]

        # Lemmatize using spaCy
        lemmatized_tokens = [token.lemma_ for token in nlp(" ".join(tokens)).doc]

        return lemmatized_tokens

    else:
        return []  # Return an empty list if the text is not a valid string

In [ ]:
%%time

processed_text_1 = df1_combined.apply(preprocess)
processed_text_2 = df2_combined.apply(preprocess)
processed_text_3 = df3_combined.apply(preprocess)

CPU times: user 9min 41s, sys: 38.4 s, total: 10min 20s
Wall time: 10min 33s


In [ ]:
processed_text_1.head(3)

,0
0,"[independent, component, analysis, identification, artifact, magnetoencephalographic, recording, independent, component, analysis, identification, artifact, magnetoencephalographic, recording, ricardo, vigario, veikko, ousmiiki, matti, hiimiiliiinen, riitta, hari, erkki, oja, lab, computer, info, science, helsinki, university, technology, box, fin, hut, finland, ricardovigario, erkkiojahutfi, brain, research, unit, low, temperature, lab, helsinki, university, technology, box, fin, hut, finland, veikko, msh, harineurohutfi, abstract, study, application, independent, component, analysis, ica, approach, identification, possible, removal, artifact, magnetoencephalographic, meg, recording, statistical, technique, separate, component, accord, kurtosis, amplitude, distribution, time, thus, distinguish, strictly, periodical, signal, regularly, irregularly, occur, signal, many, artifact, belong, last, category, order, assess, effectiveness, method, control, artifact, produce, include, saccadic, ...]"
5,"[digital, boltzmann, vlsi, constraint, satisfaction, learn, digital, boltzmann, vlsi, constraint, satisfaction, learn, michael, murray, mingtak, leung, kan, boonyanit, kong, kritayakirana, james, burrt, gregory, wolff, takahiro, watanabe, edward, schwartz, david, storktt, allen, petersont, department, electrical, engineering, stanford, university, stanford, ricoh, california, research, center, sand, hill, road, suite, menlo, park, sun, mlcrosystem, garcia, ave, mtv, room, mountain, view, abstract, build, highspeed, digital, meanfield, boltzmann, chip, sbus, board, general, problem, constraint, satjsfaction, learn, chip, neural, processor, weight, update, processor, support, arbitrary, topology, functional, neuron, onchip, learn, theoretical, maximum, rate, connection, updatessec, recall, patternssec, typical, condition, chip, high, speed, due, parallel, computation, inner, product, ...]"
7,"[optimal, brain, damage, cun, denker, solla, optimal, brain, damage, yann, cun, john, denker, sara, sola, att, bell, laboratories, holmdel, abstract, use, informationtheoretic, idea, derive, class, practical, nearly, optimal, scheme, adapt, size, neural, network, remove, unimportant, weight, network, several, improvement, expect, well, generalization, few, training, example, require, improve, speed, learn, andor, classification, basic, idea, use, secondderivative, information, make, tradeoff, network, complexity, training, set, error, experiment, confirm, usefulness, method, realworld, application, introduction, successful, application, neural, network, learn, realworld, problem, achieve, use, highly, structure, network, rather, large, size, example, waibel, cun, application, become, complex, network, presumably, become, even, large, structured, design, tool, technique, ...]"


# Generate Gensim Dictionary Object

In [ ]:
dictionary_1 = Dictionary(processed_text_1)
dictionary_2 = Dictionary(processed_text_2)
dictionary_3 = Dictionary(processed_text_3)

In [ ]:
print(f'{len(dictionary_1)} words in dictionary object 1')
print(f'{len(dictionary_2)} words in dictionary object 2')
print(f'{len(dictionary_3)} words in dictionary object 3')

29392 words in dictionary object 1
36216 words in dictionary object 2
64645 words in dictionary object 3


In [ ]:
dictionary_1.filter_extremes(no_below=10, no_above=0.6)
dictionary_2.filter_extremes(no_below=10, no_above=0.6)
dictionary_3.filter_extremes(no_below=10, no_above=0.6)

print("{} words remaining in the Dictionary 1 object.".format(len(dictionary_1)))
print("{} words remaining in the Dictionary 2 object.".format(len(dictionary_2)))
print("{} words remaining in the Dictionary 3 object.".format(len(dictionary_3)))

2647 words remaining in the Dictionary 1 object.
3400 words remaining in the Dictionary 2 object.
6445 words remaining in the Dictionary 3 object.


In [ ]:
bow_corpus_1 = [dictionary_1.doc2bow(doc) for doc in processed_text_1]
bow_corpus_2 = [dictionary_2.doc2bow(doc) for doc in processed_text_2]
bow_corpus_3 = [dictionary_3.doc2bow(doc) for doc in processed_text_3]

# Train LDA Model (BOW Corpus)

In [ ]:
lda_model_bow_1 = LdaMulticore(corpus=bow_corpus_1, id2word=dictionary_1, num_topics=10,
                             chunksize=10, passes=10, gamma_threshold=0.001,
                             per_word_topics=True, workers=multiprocessing.cpu_count(), random_state=42)

lda_model_bow_2 = LdaMulticore(corpus=bow_corpus_2, id2word=dictionary_2, num_topics=10,
                             chunksize=10, passes=10, gamma_threshold=0.001,
                             per_word_topics=True, workers=multiprocessing.cpu_count(), random_state=42)

lda_model_bow_3 = LdaMulticore(corpus=bow_corpus_3, id2word=dictionary_3, num_topics=10,
                             chunksize=10, passes=10, gamma_threshold=0.001,
                             per_word_topics=True, workers=multiprocessing.cpu_count(), random_state=42)

# Evaluate LDA Model (BOW Corpus)

In [ ]:
coherence_model_lda_1 = CoherenceModel(model=lda_model_bow_1, texts=processed_text_1,
                                     dictionary=dictionary_1, coherence='c_v')
coherence_score_1 = coherence_model_lda_1.get_coherence()
print('LDA BOW 1 Coherence Score (c_v): ', coherence_score_1)


coherence_model_lda_2 = CoherenceModel(model=lda_model_bow_2, texts=processed_text_2,
                                     dictionary=dictionary_2, coherence='c_v')
coherence_score_2 = coherence_model_lda_2.get_coherence()
print('LDA BOW 2 Coherence Score (c_v): ', coherence_score_2)


coherence_model_lda_3 = CoherenceModel(model=lda_model_bow_3, texts=processed_text_3,
                                     dictionary=dictionary_3, coherence='c_v')
coherence_score_3 = coherence_model_lda_3.get_coherence()
print('LDA BOW 3 Coherence Score (c_v): ', coherence_score_3)


LDA BOW 1 Coherence Score (c_v):  0.47094891245868453
LDA BOW 2 Coherence Score (c_v):  0.47313449021287435
LDA BOW 3 Coherence Score (c_v):  0.5276578906936535


In [ ]:
print('Average LDA BOW Coherence Score :',np.mean([coherence_score_1,coherence_score_2,coherence_score_3]))

Average LDA BOW Coherence Score : 0.49058043112173744


In [ ]:
coherence_model_lda_1 = CoherenceModel(model=lda_model_bow_1, texts=processed_text_1,
                                     dictionary=dictionary_1, coherence='u_mass')
coherence_score_1 = coherence_model_lda_1.get_coherence()
print('LDA BOW 1 Coherence Score (u_mass): ', coherence_score_1)


coherence_model_lda_2 = CoherenceModel(model=lda_model_bow_2, texts=processed_text_2,
                                     dictionary=dictionary_2, coherence='u_mass')
coherence_score_2 = coherence_model_lda_2.get_coherence()
print('LDA BOW 2 Coherence Score (u_mass): ', coherence_score_2)


coherence_model_lda_3 = CoherenceModel(model=lda_model_bow_3, texts=processed_text_3,
                                     dictionary=dictionary_3, coherence='u_mass')
coherence_score_3 = coherence_model_lda_3.get_coherence()
print('LDA BOW 3 Coherence Score (u_mass): ', coherence_score_3)

LDA BOW 1 Coherence Score (u_mass):  -1.1977206602371855
LDA BOW 2 Coherence Score (u_mass):  -1.0606618425887162
LDA BOW 3 Coherence Score (u_mass):  -0.9617021625915697


In [ ]:
print('Average LDA BOW Coherence Score (u_mass) :',np.mean([coherence_score_1,coherence_score_2,coherence_score_3]))

Average LDA BOW Coherence Score (u_mass) : -1.0733615551391573


In [ ]:
print('1987-1997 topics: ','\n')
topics_1 = lda_model_bow_1.print_topics()
for topic in topics_1:
    print(topic)

print('\n')
print('1998-2007 topics: ','\n')
topics_2 = lda_model_bow_2.print_topics()
for topic in topics_2:
    print(topic)

print('\n')
print('2008-2017 topics: ','\n')
topics_3 = lda_model_bow_3.print_topics()
for topic in topics_3:
  print(topic)

1987-1997 topics:  

(0, '0.045*"image" + 0.020*"recognition" + 0.020*"feature" + 0.017*"face" + 0.016*"object" + 0.013*"pixel" + 0.012*"detection" + 0.010*"scale" + 0.010*"ensemble" + 0.010*"level"')
(1, '0.015*"distribution" + 0.014*"probability" + 0.010*"estimate" + 0.010*"gaussian" + 0.009*"noise" + 0.008*"mixture" + 0.008*"sample" + 0.007*"density" + 0.007*"variance" + 0.007*"variable"')
(2, '0.024*"cell" + 0.020*"neuron" + 0.015*"signal" + 0.012*"stimulus" + 0.012*"response" + 0.011*"field" + 0.011*"visual" + 0.010*"activity" + 0.010*"frequency" + 0.009*"cortex"')
(3, '0.013*"let" + 0.012*"state" + 0.012*"theorem" + 0.012*"size" + 0.011*"bind" + 0.010*"threshold" + 0.010*"bound" + 0.008*"proof" + 0.006*"probability" + 0.006*"sequence"')
(4, '0.029*"classifier" + 0.020*"region" + 0.019*"feature" + 0.018*"map" + 0.015*"layer" + 0.015*"fig" + 0.013*"risk" + 0.012*"class" + 0.012*"code" + 0.011*"transformation"')
(5, '0.057*"state" + 0.035*"action" + 0.025*"task" + 0.018*"policy" + 0

# Labeling Topics

Finding Labels For Each Topic Within The Decade

In [ ]:
import re
from keybert import KeyBERT

kw_model = KeyBERT()

#Extracting Labels for 1987-1996
topic_labels_1987= []

for topic_num, topic_str in topics_1:
    # Extract words
    words = re.findall(r'"(.*?)"', topic_str)
    joined_words = " ".join(words)
    # Extract keyword with KeyBERT
    label = kw_model.extract_keywords(joined_words, keyphrase_ngram_range=(1, 3), top_n=1,use_mmr=True,diversity=.7)

    # Decide what to store
    if label:
        keyword, score = label[0]
        topic_labels_1987.append((topic_num, keyword, round(score * 100, 2)))  # percent format
    else:
        topic_labels_1987.append((topic_num, "not found", 0.0))


  #Extracting Labels for 1997-2006
topic_labels_1998= []

for topic_num, topic_str in topics_2:
    # Extract words
    words = re.findall(r'"(.*?)"', topic_str)
    joined_words = " ".join(words)
    # Extract keyword with KeyBERT
    label = kw_model.extract_keywords(joined_words, keyphrase_ngram_range=(1, 3), top_n=1,use_mmr=True,diversity=.7)

    # Decide what to store
    if label:
        keyword, score = label[0]
        topic_labels_1998.append((topic_num, keyword, round(score * 100, 2)))  # percent format
    else:
        topic_labels_1998.append((topic_num, "not found", 0.0))


  #Extracting Labels for 2007-2017
topic_labels_2008= []

for topic_num, topic_str in topics_3:
    # Extract words
    words = re.findall(r'"(.*?)"', topic_str)
    joined_words = " ".join(words)
    # Extract keyword with KeyBERT
    label = kw_model.extract_keywords(joined_words, keyphrase_ngram_range=(1, 3), top_n=1,use_mmr=True,diversity=.7)

    # Decide what to store
    if label:
        keyword, score = label[0]
        topic_labels_2008.append((topic_num, keyword, round(score * 100, 2)))  # percent format
    else:
        topic_labels_2008.append((topic_num, "not found", 0.0))

In [ ]:
print('Topic Labels For 1987-1996:\n')
for topic_num, keyword, percent in topic_labels_1987:
    print(f"Topic {topic_num+1}: {keyword} ({percent}%)")

print("\n")

print('Topic Labels For 1998-2007:\n')
for topic_num, keyword, percent in topic_labels_1998:
    print(f"Topic {topic_num+1}: {keyword} ({percent}%)")

print("\n")

print('Topic Labels For 2008-2017:\n')
for topic_num, keyword, percent in topic_labels_2008:
    print(f"Topic {topic_num+1}: {keyword} ({percent}%)")

Topic Labels For 1987-1996:

Topic 1: recognition feature face (67.37%)
Topic 2: gaussian noise mixture (78.82%)
Topic 3: activity frequency cortex (70.83%)
Topic 4: bound proof probability (63.18%)
Topic 5: fig risk class (62.14%)
Topic 6: reinforcement expert update (77.02%)
Topic 7: state acoustic mlp (70.68%)
Topic 8: memory trajectory equation (69.24%)
Topic 9: neuron chip voltage (74.9%)
Topic 10: hide net layer (69.56%)


Topic Labels For 1998-2007:

Topic 1: spatial signal motion (67.56%)
Topic 2: node tree bayesian (82.13%)
Topic 3: basis code pixel (66.4%)
Topic 4: label document manifold (66.8%)
Topic 5: approximation loss gaussian (58.86%)
Topic 6: regression margin classifier (71.51%)
Topic 7: cluster representation causal (72.44%)
Topic 8: observation belief reinforcement (81.36%)
Topic 9: neuron spike response (69.71%)
Topic 10: face boost hypothesis (67.58%)


Topic Labels For 2008-2017:

Topic 1: bayesian variational gradient (78.65%)
Topic 2: topic cluster word (77.39

Find Overarching Topic For The Decade


In [ ]:
#Combining all topic labels into one string to be fed into keyBERT
label_1987=""

for i in topic_labels_1987:
  label_1987+=i[1]+" "

label_1998=""

for i in topic_labels_1998:
  label_1998+=i[1]+" "

label_2008=""

for i in topic_labels_2008:
  label_2008+=i[1]+" "


label_1987 = kw_model.extract_keywords(label_1987, keyphrase_ngram_range=(1, 4), top_n=1)

label_1998 = kw_model.extract_keywords(label_1998, keyphrase_ngram_range=(1, 4), top_n=1)

label_2008 = kw_model.extract_keywords(label_2008, keyphrase_ngram_range=(1, 4), top_n=1)

In [ ]:
print('Overaching Topic Label For 1987-1997:',label_1987[0][0],f'({round(label_1987[0][1]*100,2)}%)\n')

print('Overaching Topic Label For 1998-2008:',label_1998[0][0],f'({round(label_1998[0][1]*100,2)}%)\n')

print('Overaching Topic Label For 2008-2017:',label_2008[0][0],f'({round(label_2008[0][1]*100,2)}%)\n')

Overaching Topic Label For 1987-1997: mixture activity frequency cortex (52.14%)

Overaching Topic Label For 1998-2008: margin classifier cluster representation (52.65%)

Overaching Topic Label For 2008-2017: variational gradient topic cluster (56.42%)



# LDA Model (BOW Corpus) Visualization

In [ ]:
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(lda_model_bow_1, bow_corpus_1, dictionary_1)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.078434 -0.099979       1        1  20.786331
2      0.033671  0.030449       2        1  16.254071
9     -0.029094  0.015727       3        1  14.484329
7     -0.005707 -0.092914       4        1  11.986911
3      0.039014 -0.154293       5        1   9.313741
0     -0.109362  0.166239       6        1   7.994716
4     -0.079291 -0.009201       7        1   5.773015
5     -0.037189 -0.150371       8        1   4.996283
6     -0.093702  0.213870       9        1   4.682313
8      0.360095  0.080474      10        1   3.728289, topic_info=             Term         Freq        Total Category  logprob  loglift
609         state  1576.000000  1576.000000  Default  30.0000  30.0000
1637        image   969.000000   969.000000  Default  29.0000  29.0000
1755         word   768.000000   768.000000  Default  28.0000  28.0000
842   recognition   978.000000   978.000000  Default  27.0000  27.0000
537        neuron  1225.000000  1225.000000  Default  26.0000  26.0000
...           ...          ...          ...      ...      ...      ...
543      parallel    92.537770   312.701797  Topic10  -4.6668   2.0716
1036     synaptic    71.380167   221.234291  Topic10  -4.9264   2.1580
609         state   103.102313  1576.661552  Topic10  -4.5587   0.5619
356         allow    80.881909   492.116157  Topic10  -4.8015   1.4835
402       compute    75.047126   655.603665  Topic10  -4.8763   1.1218

[630 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
2394      1  0.075344          acm
2394      5  0.904126          acm
1678      9  0.991991     acoustic
1053      2  0.066216       action
1053      4  0.019475       action
...     ...       ...          ...
1755      5  0.010409         word
1755      6  0.018216         word
1755      9  0.938142         word
2641      1  0.080278  workstation
2641      3  0.883057  workstation

[1787 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 3, 10, 8, 4, 1, 5, 6, 7, 9])

In [ ]:
pyLDAvis.gensim.prepare(lda_model_bow_2, bow_corpus_2, dictionary_2)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.084804 -0.030388       1        1  15.042434
4      0.141810  0.020719       2        1  13.883015
5      0.137971  0.092233       3        1  13.271014
8     -0.092551 -0.117785       4        1  12.704224
2     -0.030083  0.021575       5        1  12.538098
3      0.014407  0.167268       6        1   7.929777
0     -0.237084 -0.103354       7        1   7.250937
9     -0.030187 -0.006114       8        1   6.715732
7      0.149199 -0.196680       9        1   5.832361
6     -0.138288  0.152525      10        1   4.832407, topic_info=           Term         Freq        Total Category  logprob  loglift
469       state  2354.000000  2354.000000  Default  30.0000  30.0000
1618     kernel  2544.000000  2544.000000  Default  29.0000  29.0000
1031    cluster  1501.000000  1501.000000  Default  28.0000  28.0000
229       image  2597.000000  2597.000000  Default  27.0000  27.0000
1141     action   865.000000   865.000000  Default  26.0000  26.0000
...         ...          ...          ...      ...      ...      ...
862    distance   107.783911   909.721564  Topic10  -5.2735   0.8968
734   selection   103.946757   684.746165  Topic10  -5.3098   1.1447
354     pattern   100.292586   915.221445  Topic10  -5.3456   0.8188
1136     weight   100.756701  1317.437627  Topic10  -5.3410   0.4591
185         fig    97.239713   908.262005  Topic10  -5.3765   0.7955

[687 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
3336      1  0.976945          
2731      1  0.991349  abstraction
1917      6  0.954986          acl
1141      4  0.023102       action
1141      9  0.976069       action
...     ...       ...          ...
1001      2  0.012048         word
1001      6  0.760253         word
1001     10  0.062652         word
2779      2  0.987522    worstcase
2987      5  0.979066      yedidia

[1838 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 5, 6, 9, 3, 4, 1, 10, 8, 7])

In [ ]:
pyLDAvis.gensim.prepare(lda_model_bow_3, bow_corpus_3, dictionary_3)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
9     -0.074823 -0.040975       1        1  15.149582
5      0.165229 -0.023195       2        1  15.115481
2     -0.180605 -0.076853       3        1  14.867563
0     -0.014217 -0.024707       4        1  12.306684
3      0.037114 -0.026547       5        1  11.288303
7      0.054164 -0.070093       6        1   7.393367
8     -0.069164 -0.025852       7        1   7.103315
6     -0.189145  0.018381       8        1   6.748028
4      0.235376 -0.083930       9        1   5.793303
1      0.036071  0.353771      10        1   4.234375, topic_info=                Term         Freq        Total Category  logprob  loglift
272            image  5896.000000  5896.000000  Default  30.0000  30.0000
325            label  4533.000000  4533.000000  Default  29.0000  29.0000
1709           graph  3384.000000  3384.000000  Default  28.0000  28.0000
969            topic  2506.000000  2506.000000  Default  27.0000  27.0000
1365         cluster  2946.000000  2946.000000  Default  26.0000  26.0000
...              ...          ...          ...      ...      ...      ...
375           metric   261.351687  1560.475042  Topic10  -5.2364   1.3751
413    nonparametric   221.226269   696.334641  Topic10  -5.4031   2.0153
1194          online   253.411976  2278.693392  Topic10  -5.2673   0.9656
2062       inference   247.473769  3455.805244  Topic10  -5.2910   0.5254
501   representation   221.765266  2368.039156  Topic10  -5.4007   0.7937

[692 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
2888      6  0.991297     abbeel
660       1  0.033808   accuracy
660       2  0.211057   accuracy
660       3  0.005796   accuracy
660       4  0.081139   accuracy
...     ...       ...        ...
649       8  0.036365  worstcase
5310      7  0.988807        www
5172      7  0.989811      yahoo
3228      8  0.983654    yedidia
6094      5  0.991708        yij

[1755 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[10, 6, 3, 1, 4, 8, 9, 7, 5, 2])

## Run TF-IDF on Bag of Words

In [ ]:
tfidf_model_1 = TfidfModel(bow_corpus_1)
tfidf_model_2 = TfidfModel(bow_corpus_2)
tfidf_model_3 = TfidfModel(bow_corpus_3)

In [ ]:
tfidf_corpus_1 = tfidf_model_1[bow_corpus_1]
tfidf_corpus_2 = tfidf_model_2[bow_corpus_2]
tfidf_corpus_3 = tfidf_model_3[bow_corpus_3]

# Train LDA Model (TF-IDF Corpus)

In [ ]:
lda_model_tfidf_1 = LdaMulticore(corpus=tfidf_corpus_1, id2word=dictionary_1, num_topics=10,
                               chunksize=10, passes=10, gamma_threshold=0.001,
                               per_word_topics=True, workers=multiprocessing.cpu_count(), random_state=42)

lda_model_tfidf_2 = LdaMulticore(corpus=tfidf_corpus_2, id2word=dictionary_2, num_topics=10,
                               chunksize=10, passes=10, gamma_threshold=0.001,
                               per_word_topics=True, workers=multiprocessing.cpu_count(), random_state=42)

lda_model_tfidf_3 = LdaMulticore(corpus=tfidf_corpus_3, id2word=dictionary_3, num_topics=10,
                               chunksize=10, passes=10, gamma_threshold=0.001,
                               per_word_topics=True, workers=multiprocessing.cpu_count(), random_state=42)

# Evaluate LDA Model (TF-IDF Corpus)

In [ ]:
coherence_model_lda_1 = CoherenceModel(model=lda_model_tfidf_1, texts=processed_text_1,
                                     dictionary=dictionary_1, coherence='c_v')
coherence_score_1 = coherence_model_lda_1.get_coherence()
print('LDA TF-IDF 1 Coherence Score (c_v): ', coherence_score_1)



coherence_model_lda_2 = CoherenceModel(model=lda_model_tfidf_2, texts=processed_text_2,
                                     dictionary=dictionary_2, coherence='c_v')
coherence_score_2 = coherence_model_lda_2.get_coherence()
print('LDA TF-IDF 2 Coherence Score (c_v): ', coherence_score_2)



coherence_model_lda_3 = CoherenceModel(model=lda_model_tfidf_3, texts=processed_text_3,
                                     dictionary=dictionary_3, coherence='c_v')
coherence_score_3 = coherence_model_lda_3.get_coherence()
print('LDA TF-IDF 3 Coherence Score (c_v): ', coherence_score_3)

LDA TF-IDF 1 Coherence Score (c_v):  0.42349728015790616
LDA TF-IDF 2 Coherence Score (c_v):  0.5911043231854463
LDA TF-IDF 3 Coherence Score (c_v):  0.4879987419513928


In [ ]:
print('Average LDA TF-IDF Coherence Score (c_v): ',np.mean([coherence_score_1,coherence_score_2,coherence_score_3]))

Average LDA TF-IDF Coherence Score (c_v):  0.5008667817649152


In [ ]:
coherence_model_lda_1 = CoherenceModel(model=lda_model_tfidf_1, texts=processed_text_1,
                                     dictionary=dictionary_1, coherence='u_mass')
coherence_score_1 = coherence_model_lda_1.get_coherence()
print('LDA TF-IDF 1 Coherence Score (u_mass): ', coherence_score_1)



coherence_model_lda_2 = CoherenceModel(model=lda_model_tfidf_2, texts=processed_text_2,
                                     dictionary=dictionary_2, coherence='u_mass')
coherence_score_2 = coherence_model_lda_2.get_coherence()
print('LDA TF-IDF 2 Coherence Score (u_mass): ', coherence_score_2)



coherence_model_lda_3 = CoherenceModel(model=lda_model_tfidf_3, texts=processed_text_3,
                                     dictionary=dictionary_3, coherence='u_mass')
coherence_score_3 = coherence_model_lda_3.get_coherence()
print('LDA TF-IDF 3 Coherence Score (u_mass): ', coherence_score_3)

LDA TF-IDF 1 Coherence Score (u_mass):  -7.640584785707384
LDA TF-IDF 2 Coherence Score (u_mass):  -11.450231554297142
LDA TF-IDF 3 Coherence Score (u_mass):  -10.311894908443017


In [ ]:
print('Average LDA TF-IDF Coherence Score (u_mass): ',np.mean([coherence_score_1,coherence_score_2,coherence_score_3]))

Average LDA TF-IDF Coherence Score (u_mass):  -9.800903749482515
